### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
model_data.shape

(478328, 2831)

### Drop Null target records

In [4]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [5]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [6]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [7]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [11]:
features_list = [
'num_PERSONAL LOAN_enq_last_6M',
'num_CONSUMER LOAN_enq_last_6M_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'num_CREDIT CARD_enq_last_12M',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'Utilization_CREDIT CARD_opened_last_36M',
'total_emi',
'total_credit_all_open',
    
# banking
'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',

#new banking features
'ratio_balance_after_2_days_max_credit_M3',
'ratio_5th_day_balance_to_max_credit_M3',
                 
#Loan application Features
'deviation_from_median_income/standard_deviation_income#last_365_Days',
'loan_amount_to_income#last_365_Days',
    
# New BUreau features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
          
'month_year',
'60_in_15mob']

In [12]:
len(features_list)

28

In [13]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [14]:
oot_df = oot_df1[features_list]

In [15]:
model_df.shape, oot_df.shape

((101984, 28), (17343, 28))

In [16]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [17]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [18]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [19]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [20]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3045530/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [21]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3045530/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [22]:
X_oot.shape,y_oot.shape

((17343, 26), (17343,))

### Split Model dataset into Train and Test

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [24]:
X_train.shape, X_test.shape

((61190, 26), (40794, 26))

In [25]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 26) (12238, 26) (40794, 26)


In [26]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [34]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.4,
    'colsample_bytree': 0.4,

    'lambda': 6.8,
    'alpha': 4.6,
    'gamma': 6.2,
   
     'scale_pos_weight':3,
    'random_state': 24
    
}



evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 700,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.59600	eval-auc:0.58794
[10]	train-auc:0.64964	eval-auc:0.64343
[20]	train-auc:0.65051	eval-auc:0.64229
[30]	train-auc:0.65343	eval-auc:0.64604
[40]	train-auc:0.65467	eval-auc:0.64870
[50]	train-auc:0.65566	eval-auc:0.64947
[60]	train-auc:0.65724	eval-auc:0.65021
[70]	train-auc:0.65866	eval-auc:0.65145
[80]	train-auc:0.65942	eval-auc:0.65269
[90]	train-auc:0.66000	eval-auc:0.65271
[100]	train-auc:0.66130	eval-auc:0.65393
[110]	train-auc:0.66207	eval-auc:0.65468
[120]	train-auc:0.66279	eval-auc:0.65490
[130]	train-auc:0.66337	eval-auc:0.65482
[140]	train-auc:0.66419	eval-auc:0.65548
[150]	train-auc:0.66480	eval-auc:0.65632
[160]	train-auc:0.66557	eval-auc:0.65685
[170]	train-auc:0.66612	eval-auc:0.65714
[180]	train-auc:0.66679	eval-auc:0.65767
[190]	train-auc:0.66776	eval-auc:0.65848
[200]	train-auc:0.66805	eval-auc:0.65841
[210]	train-auc:0.66865	eval-auc:0.65891
[220]	train-auc:0.66915	eval-auc:0.65934
[230]	train-auc:0.66991	eval-auc:0.65993
[240]	train-auc:0.67063	eva

In [35]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.69, Gini: 0.38, KS: 0.28
Eval AUC: 0.67, Gini: 0.35, KS: 0.25
Test AUC: 0.67, Gini: 0.34, KS: 0.25
OOT AUC: 0.69, Gini: 0.37, KS: 0.28


In [36]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.69, Gini: 0.3822, KS: 0.2799
Test AUC: 0.67, Gini: 0.3416, KS: 0.2468
OOT AUC: 0.69, Gini: 0.3713, KS: 0.2772


In [37]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
24                          Num_Enq_non_BANK_last_12M       209.0
2              Num_of_PERSONAL LOAN_opened_last6M_12M       174.0
23                           No of enq in past 90days       155.0
5                highest_sanction_amount_credit_cards       155.0
6                          Age_of_oldest_open_account       123.0
1                   num_CONSUMER LOAN_enq_last_6M_12M       105.0
7             Utilization_CREDIT CARD_opened_last_36M        96.0
18  deviation_from_median_income/standard_deviatio...        93.0
8                                           total_emi        90.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...        81.0
22                       percentage_utilization_gt_75        80.0
11                           Min_Balance_Min_M1_M2_M3        80.0
20                         percentage_ever_delinquent        78.0
10                         bounced_transactions_count        76.0
19        

In [38]:
feature_importance_df

Feature  Importance
24                          Num_Enq_non_BANK_last_12M       209.0
2              Num_of_PERSONAL LOAN_opened_last6M_12M       174.0
23                           No of enq in past 90days       155.0
5                highest_sanction_amount_credit_cards       155.0
6                          Age_of_oldest_open_account       123.0
1                   num_CONSUMER LOAN_enq_last_6M_12M       105.0
7             Utilization_CREDIT CARD_opened_last_36M        96.0
18  deviation_from_median_income/standard_deviatio...        93.0
8                                           total_emi        90.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...        81.0
22                       percentage_utilization_gt_75        80.0
11                           Min_Balance_Min_M1_M2_M3        80.0
20                         percentage_ever_delinquent        78.0
10                         bounced_transactions_count        76.0
19                loan_amount_to_income#last_365_Days        57.0
3                        num_CREDIT CARD_enq_last_12M        53.0
21   months_since_most_recent_delinquency_installment        52.0
9                               total_credit_all_open        50.0
25                     Num_of_installment_loan_opened        48.0
12                               num_credit_gt_salary        48.0
17             ratio_5th_day_balance_to_max_credit_M3        41.0
15               ratio_total_debit_to_total_credit_M3        35.0
16           ratio_balance_after_2_days_max_credit_M3        34.0
14                            Max_Credit_Avg_M1_M2_M3        27.0
0                       num_PERSONAL LOAN_enq_last_6M        24.0
13                                    upi_trans_count        23.0

In [39]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.273897         0.482698        0.750192        0.394648   4896   
9          0.174464         0.355462        0.394630        0.324887   4895   
8          0.138100         0.302986        0.324885        0.283829   4895   
7          0.107252         0.268117        0.283820        0.253136   4895   
6          0.092952         0.239910        0.253136        0.227643   4895   
5          0.087436         0.216803        0.227638        0.206521   4895   
4          0.069867         0.197018        0.206506        0.187747   4895   
3          0.057610         0.179054        0.187744        0.170467   4895   
2          0.045148         0.161198        0.170465        0.149629   4895   
1          0.032271         0.131983        0.149622        0.088553   4896   

        label_good  label_bad  
Decile                         
10            3555       1341  
9             4041        854  
8             4219        676  
7             4370        525  
6             4440        455  
5             4467        428  
4             4553        342  
3             4613        282  
2             4674        221  
1             4738        158

In [40]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.266667         0.480475        0.743944        0.395226   4080   
9          0.162050         0.355607        0.395215        0.324368   4079   
8          0.139250         0.302187        0.324346        0.283063   4079   
7          0.114461         0.267235        0.283059        0.252348   4080   
6          0.096102         0.239853        0.252326        0.227992   4079   
5          0.088502         0.217381        0.227987        0.207000   4079   
4          0.083824         0.197169        0.206997        0.187819   4080   
3          0.066193         0.179137        0.187814        0.170603   4079   
2          0.050503         0.161459        0.170598        0.150216   4079   
1          0.038725         0.132551        0.150214        0.092104   4080   

        label_good  label_bad  
Decile                         
10            2992       1088  
9             3418        661  
8             3511        568  
7             3613        467  
6             3687        392  
5             3718        361  
4             3738        342  
3             3809        270  
2             3873        206  
1             3922        158

In [41]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.270317         0.492175        0.729813        0.401887   1735   
9          0.151672         0.361531        0.401874        0.329490   1734   
8          0.143599         0.305409        0.329480        0.284431   1734   
7          0.117070         0.267595        0.284430        0.252256   1734   
6          0.091119         0.239716        0.252244        0.227517   1734   
5          0.076081         0.216611        0.227503        0.206409   1735   
4          0.069781         0.197197        0.206409        0.188083   1734   
3          0.057093         0.179450        0.188067        0.171359   1734   
2          0.049596         0.162615        0.171358        0.151856   1734   
1          0.033429         0.133182        0.151848        0.090477   1735   

        label_good  label_bad  
Decile                         
10            1266        469  
9             1471        263  
8             1485        249  
7             1531        203  
6             1576        158  
5             1603        132  
4             1613        121  
3             1635         99  
2             1648         86  
1             1677         58

In [85]:
xgb_model.save_model('LeoPayu_BBLA_60_in_15mob.json')
print('Model Saved')

Model Saved


In [42]:
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBL_Train_Pred.csv',index=False)
Eval.to_csv('LeoPayu_BBL_Eval_Pred.csv',index=False)
Test.to_csv('LeoPayu_BBL_Test_Pred.csv',index=False)
Oot.to_csv('LeoPayu_BBL_Oot_Pred.csv',index=False)
